In [2]:
import pandas as pd

df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [4]:
df.koi_disposition.unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [5]:
# remove space for false positive in koi_dispotion
mask = df['koi_disposition'] == "FALSE POSITIVE"
df.loc[mask, 'koi_disposition'] = "FALSE_POSITIVE"
df.koi_disposition.unique()

array(['CONFIRMED', 'FALSE_POSITIVE', 'CANDIDATE'], dtype=object)

In [7]:
# create train and test sets
from sklearn.model_selection import train_test_split

y = df['koi_disposition']
X = df.drop('koi_disposition', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [8]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
5964,1,0,0,0,252.044440,0.027490,-0.027490,265.2010,0.0494,-0.0494,...,-136.0,4.621,0.041,-0.035,0.664,0.057,-0.059,292.79022,41.948639,15.884
9410,1,0,0,0,371.518520,0.015790,-0.015790,317.6836,0.0339,-0.0339,...,-206.0,4.377,0.101,-0.203,1.089,0.364,-0.145,293.06400,45.034210,13.731
4204,0,0,1,0,8.038670,0.000114,-0.000114,135.3098,0.0123,-0.0123,...,-181.0,4.485,0.050,-0.200,0.975,0.282,-0.101,290.51785,41.238762,14.999
5933,0,0,0,0,18.782160,0.000406,-0.000406,147.8508,0.0148,-0.0148,...,-167.0,4.488,0.048,-0.290,0.940,0.386,-0.087,291.76413,41.860130,14.043
6996,1,0,1,0,12.429716,0.000472,-0.000472,141.2846,0.0420,-0.0420,...,-200.0,4.534,0.037,-0.213,0.905,0.281,-0.088,297.52072,40.585419,15.842


In [9]:
# preprocessing
# create scalers

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.layers import Dense

In [13]:
# scale with labelencoders
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [14]:
y_train_categorical.shape

(6558, 3)

In [15]:
# create deep learning model
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
# compile model
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [18]:
# fit the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
 - 2s - loss: 0.5184 - acc: 0.7324
Epoch 2/60
 - 1s - loss: 0.3660 - acc: 0.8082
Epoch 3/60
 - 1s - loss: 0.3492 - acc: 0.8243
Epoch 4/60
 - 1s - loss: 0.3471 - acc: 0.8292
Epoch 5/60
 - 1s - loss: 0.3322 - acc: 0.8435
Epoch 6/60
 - 1s - loss: 0.3313 - acc: 0.8455
Epoch 7/60
 - 1s - loss: 0.3248 - acc: 0.8437
Epoch 8/60
 - 1s - loss: 0.3173 - acc: 0.8463
Epoch 9/60
 - 1s - loss: 0.3107 - acc: 0.8568
Epoch 10/60
 - 1s - loss: 0.3061 - acc: 0.8646
Epoch 11/60
 - 1s - loss: 0.3010 - acc: 0.8651
Epoch 12/60
 - 1s - loss: 0.3029 - acc: 0.8594
Epoch 13/60
 - 1s - loss: 0.2969 - acc: 0.8644
Epoch 14/60
 - 1s - loss: 0.2958 - acc: 0.8647
Epoch 15/60
 - 1s - loss: 0.2894 - acc: 0.8683
Epoch 16/60
 - 1s - loss: 0.2818 - acc: 0.8762
Epoch 17/60
 - 1s - loss: 0.2834 - acc: 0.8725
Epoch 18/60
 - 1s - loss: 0.2811 - acc: 0.8739
Epoch 19/60
 - 1s - loss: 0.2742 - acc: 0.8809
Epoch 20/60
 - 1s - loss: 0.2743 - acc: 0.8800
Epoch 21/60
 - 1s - l

In [19]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)

print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 0.2612 - acc: 0.8902
Normal Neural Network - Loss: 0.2611865277477938, Accuracy: 0.8902104496955872


In [20]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [21]:
print(f"Predicted clases: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted clases: ['FALSE_POSITIVE' 'CONFIRMED' 'CONFIRMED' 'CANDIDATE' 'CONFIRMED'
 'FALSE_POSITIVE' 'FALSE_POSITIVE' 'FALSE_POSITIVE' 'FALSE_POSITIVE'
 'FALSE_POSITIVE']
Actual Labels: ['FALSE_POSITIVE', 'CANDIDATE', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE', 'FALSE_POSITIVE', 'FALSE_POSITIVE', 'FALSE_POSITIVE', 'FALSE_POSITIVE', 'FALSE_POSITIVE']
